## transposition.py

In [5]:
#!/usr/bin/env python
# transposition.py
import numpy as np
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank(); size = comm.Get_size(); status = MPI.Status();
N = 16
unsorted = np.zeros(N, dtype="int")
final_sorted = np.zeros(N, dtype="int")
local_array = np.zeros(int(N / size), dtype="int")
local_tmp = np.zeros(int(N / size), dtype="int")
local_remain = np.zeros(2 * int(N / size), dtype="int")

if rank == 0:
    unsorted = np.random.randint(low=0,high=N,size=N)
    print (unsorted)
comm.Scatter(unsorted, local_array, root = 0)

local_array.sort()
for step in range(0, size):
    print ("Step: ", step)
    if (step % 2 == 0):
        if (rank % 2 == 0):
            des = rank + 1
        else:
            des = rank - 1
    else:
        if (rank % 2 == 0):
            des = rank - 1
        else:
            des = rank + 1
            
    if (des >= 0 and des < size):
        print ("My rank is ", rank, " and my des is ", des)
        comm.Send(local_array, dest = des, tag = 0)
        comm.Recv(local_tmp, source = des)    
        print ("Rank ", rank, " ", step, ": Initial local_array: ", local_array)
        print ("Rank ", rank, " ", step, ": Received local_tmp:", local_tmp)
        local_remain = np.concatenate((local_array, local_tmp), axis=0)
        local_remain.sort()
        
        if (rank < des):
            local_array = local_remain[0:int(N/size)]
        else:
            local_array = local_remain[int(N/size):2 * int(N/size)]
        print ("Rank ", rank, " ", step, ": Retained portions: ", local_array)
comm.Gather(local_array, final_sorted)

if (rank  == 0):
    print (final_sorted)

[14 10 13 11  1 15 10  9  2  1  0 12 13 13 12  4]
Step:  0
[ 0  1  1  2  4  9 10 10 11 12 12 13 13 13 14 15]


In [1]:
!module load anaconda3/5.1.0-gcc; mpirun -np 4 --mca mpi_cuda_support 0 python transposition.py

[12 10 14 10  9  2  6  5 12  0  1 10  9  4  3  4]
Step:  0
My rank is  0  and my des is  1
Rank  0   0 : Initial local_array:  [10 10 12 14]
Rank  0   0 : Received local_tmp: [2 5 6 9]
Rank  0   0 : Retained portions:  [2 5 6 9]
Step:  1
Step:  2
My rank is  0  and my des is  1
Step:  0
My rank is  1  and my des is  0
Rank  1   0 : Initial local_array:  [2 5 6 9]
Rank  1   0 : Received local_tmp: [10 10 12 14]
Rank  1   0 : Retained portions:  [10 10 12 14]
Step:  1
My rank is  1  and my des is  2
Rank  1   1 : Initial local_array:  [10 10 12 14]
Rank  1   1 : Received local_tmp: [0 1 3 4]
Step:  0
My rank is  2  and my des is  3
Rank  2   0 : Initial local_array:  [ 0  1 10 12]
Rank  2   0 : Received local_tmp: [3 4 4 9]
Rank  2   0 : Retained portions:  [0 1 3 4]
Step:  1
My rank is  2  and my des is  1
Rank  2   1 : Initial local_array:  [0 1 3 4]
Rank  2   1 : Received local_tmp: [10 10 12 14]
Step:  0
My rank is  3  and my des is  2
Rank  3   0 : Initial local_array:  [3 4 4 9]
Ra

## merge.py

In [5]:
#!/usr/bin/env python
# merge.py
import numpy as np
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank(); size = comm.Get_size(); status = MPI.Status();
N = 16
unsorted = np.zeros(N, dtype="int")
final_sorted = np.zeros(N, dtype="int")
local_array = np.zeros(int(N / size), dtype="int")
local_tmp = np.zeros(int(N / size), dtype="int")
local_remain = np.zeros(2 * int(N / size), dtype="int")

if rank == 0:
    unsorted = np.random.randint(low=0,high=N,size=N)
    print (unsorted)
comm.Scatter(unsorted, local_array, root = 0)

local_array.sort()

step = size / 2
print ("Rank: ", rank)
while (step >= 1):
    if (rank >= step and rank < step * 2):
        comm.Send(local_array, rank - step, tag = 0)
    elif (rank < step):
        local_tmp = np.zeros(local_array.size, dtype="int")
        local_remain = np.zeros(2 * local_array.size, dtype="int")
        comm.Recv(local_tmp, rank + step, tag = 0)
        i = 0 #local_array counter
        j = 0 # local_tmp counter
        for k in range (0, 2 * local_array.size):
            if (i >= local_array.size):
                local_remain[k] = local_tmp[j]
                j += 1
            elif (j >= local_array.size):
                local_remain[k] = local_array[i]
                i += 1
            elif (local_array[i] > local_tmp[j]):
                local_remain[k] = local_tmp[j]
                j += 1
            else:
                local_remain[k] = local_array[i]
                i += 1        
        print ("Step: ", step)
        print ("local array: ", local_array)
        print ("local tmp: ", local_tmp)
        print ("local remain: ", local_remain)
        local_array = local_remain
    step = step / 2
    
if (rank  == 0):
    print (local_array)

[ 5 13  2 12 10 15  2 11 11 15  2 14 14  8  6 11]
Rank:  0
[ 2  2  2  5  6  8 10 11 11 11 12 13 14 14 15 15]


In [8]:
!module load anaconda3/5.1.0-gcc; mpirun -np 4 --mca mpi_cuda_support 0 python merge.py

[ 8  1  1  9  8  0  5  8  7 15  1  8 12 11  6  5]
Rank:  0
Rank:  1
Rank:  2
Rank:  3
Step:  2.0
Step:  2.0
local array:  [1 1 8 9]
local tmp:  [ 1  7  8 15]
local array:  [0 5 8 8]
local remain:  [ 1  1  1  7  8  8  9 15]
local tmp:  [ 5  6 11 12]
local remain:  [ 0  5  5  6  8  8 11 12]
Step:  1.0
local array:  [ 1  1  1  7  8  8  9 15]
local tmp:  [ 0  5  5  6  8  8 11 12]
local remain:  [ 0  1  1  1  5  5  6  7  8  8  8  8  9 11 12 15]
[ 0  1  1  1  5  5  6  7  8  8  8  8  9 11 12 15]


## quicksort.py

In [3]:
#!/usr/bin/env python
# quicksort.py
import numpy as np
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank(); size = comm.Get_size(); status = MPI.Status();
N = 16
HAS = 1
HASNOT = 0
unsorted = np.zeros(N, dtype="int")
final_sorted = np.zeros(N, dtype="int")
local_array = None
local_tmp = None
local_tmp_size = np.zeros(1,dtype="int")

if rank == 0:
    unsorted = np.random.randint(low=0,high=N,size=N)
    print ("Unsorted array ", unsorted)
    local_array = unsorted

distance = size / 2
print ("Rank: ", rank)
while (distance >= 1):
    if (rank % distance == 0 and (rank / distance) % 2 == 0):
        print ("Rank ", rank, " send to rank ", int(rank + distance))
        if (local_array is not None):
            if local_array.size == 1 or np.unique(local_array).size == 1:
                comm.Send(local_array[0], dest = rank + distance, tag = HASNOT)
            else:
            #    print ("median is ", np.median(local_array))
                local_tmp = local_array[local_array > np.median(local_array)]
                comm.Send(np.full(shape = 1, fill_value = local_tmp.size, dtype="int"), dest = rank + distance, tag = HAS)
                comm.Send(local_tmp, dest = rank + distance, tag = HAS)
                local_array = local_array[local_array <= np.median(local_array)]
        else:
            comm.Send(np.zeros(1,dtype="int"), rank + distance, tag = HASNOT)
    elif (rank % distance == 0 and (rank / distance) % 2 == 1):
        comm.Recv(local_tmp_size, source = rank - distance, tag = MPI.ANY_TAG, status = status)
        if status.Get_tag() == HASNOT:
            continue
        else:
            local_array = np.zeros(local_tmp_size[0], dtype="int")
            comm.Recv(local_array, source = rank - distance, tag = MPI.ANY_TAG, status = status)
    distance /= 2
#    print (local_array)
    
local_array.sort()
print ("Local array at rank ", rank, ": ", local_array)

Unsorted array  [ 0  6  1  3  4  1 13  7 14 12  2  2 15  5  2  2]
Rank:  0
Local array at rank  0 :  [ 0  1  1  2  2  2  2  3  4  5  6  7 12 13 14 15]


In [9]:
!module load anaconda3/5.1.0-gcc; mpirun -np 8 --mca mpi_cuda_support 0 python quicksort.py

Rank:  2
Rank  2  send to rank  3
Local array at rank  2 :  [5]
Rank:  3
Local array at rank  3 :  [9]
Rank:  4
Rank  4  send to rank  6
Rank  4  send to rank  5
Local array at rank  4 :  [10 10 11]
Rank:  5
Local array at rank  5 :  [12 12]
Rank:  6
Rank  6  send to rank  7
Local array at rank  6 :  [14 15 15]
Rank:  7
Local array at rank  7 :  []
Unsorted array  [ 9 12  5 14  0  3 10  4 15 11 15  0  4 10  4 12]
Rank:  0
Rank  0  send to rank  4
Rank  0  send to rank  2
Rank  0  send to rank  1
Local array at rank  0 :  [0 0 3]
Rank:  1
Local array at rank  1 :  [4 4 4]
